# Experiments

In [ ]:
#!pip install tensorflow==2.10.0
#!pip install docplex
#!pip install cplex

In [218]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

# For type annotations
import numpy as np

In [219]:
dataset_name = 'glass'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

In [220]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

In [222]:
print(mp_model.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model2

Minimize
 obj:
Subject To
 c_0_0: - 0.246800288558 x_0 + 0.373046636581 x_1 - 0.087541021407 x_2
        + 0.151911556721 x_3 - 0.188099548221 x_4 + 0.008409903385 x_5
        - 0.228263527155 x_6 - 0.314106106758 x_7 - 0.231865823269 x_8 - y_0_0
        + s_0_0 = -0.029587199911
 c_0_1: 0.296697169542 x_0 + 0.047410625964 x_1 - 0.083432085812 x_2
        + 0.265238046646 x_3 - 0.126896679401 x_4 - 0.175816029310 x_5
        + 0.201700001955 x_6 - 0.018465248868 x_7 - 0.290077596903 x_8 - y_0_1
        + s_0_1 = -0.041153822094
 c_0_2: - 0.188123106956 x_0 - 0.393000423908 x_1 - 0.325789839029 x_2
        - 0.116246759892 x_3 - 0.394245743752 x_4 + 0.188852831721 x_5
        + 0.323486834764 x_6 + 0.037820912898 x_7 + 0.446245729923 x_8 - y_0_2
        + s_0_2 = -0.163335874677
 c_0_3: 0.378312617540 x_0 - 0.398094445467 x_1 + 0.232538312674 x_2
        + 0.057570852339 x_3 + 0.133381128311 x

In [223]:
# i = 134 is also a nice value to study
i = 138
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 138
1/1 [==============================] - 0s 13ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.0007575714
Network output: (<class 'numpy.int64'>) 1


In [224]:
mdl_aux = mp_model.clone()
minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

### Improving the Explanation

In [225]:
import docplex

In [281]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

In [282]:
#print(testing_model.export_as_lp_string())

In [283]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

In [284]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [285]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,2.967691214515491),
 docplex.mp.LinearConstraint[input GE](x_0,GE,2.967691214515491),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.408120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [286]:
#print(testing_model.export_as_lp_string())

In [287]:
def log_and_improve_explanation(minimal_explanation: list, epsilon: float):
	for constraint in minimal_explanation:
		testing_model.solve()
		print('Initial constraint:' + '\t', constraint)

		variable = constraint.lhs
		while testing_model.solution is None:
			if constraint.sense == docplex.mp.constants.ComparisonType.LE:
				if constraint.rhs.constant <= variable.ub:
					constraint.rhs += epsilon
				else:
					break
			elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
				if constraint.rhs.constant >= variable.lb:
					constraint.rhs -= epsilon
				else:
					break
			else:
				raise Exception('Constraint sense was neither LE nor GE')

			testing_model.solve()

		# Undo last operation
		if constraint.sense == docplex.mp.constants.ComparisonType.LE:
			constraint.rhs -= epsilon
		elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
			constraint.rhs += epsilon

		print('Final constraint:' + '\t', constraint)
		print()

In [288]:
def find_bounds(minimal_explanation: list):
    for constraint in minimal_explanation:
        #testing_model.solve()
        print('Initial constraint:' + '\t', constraint)

        variable = constraint.lhs
        print(f"variable {variable} ub {variable.ub}")
        print(f"variable {variable} lb {variable.lb}")
find_bounds(linear_constraints)

Initial constraint:	 input LE: x_0 <= 2.967691214515491
variable x_0 ub 5.1279016612406805
variable x_0 lb -2.343651902203461
Initial constraint:	 input GE: x_0 >= 2.967691214515491
variable x_0 ub 5.1279016612406805
variable x_0 lb -2.343651902203461
Initial constraint:	 input LE: x_3 <= -1.408120229258977
variable x_3 ub 4.136918742833424
variable x_3 lb -2.335654020954433
Initial constraint:	 input GE: x_3 >= -1.408120229258977
variable x_3 ub 4.136918742833424
variable x_3 lb -2.335654020954433
Initial constraint:	 input LE: x_5 <= -0.790702170757714
variable x_5 ub 8.672524288611543
variable x_5 lb -0.790702170757714
Initial constraint:	 input GE: x_5 >= -0.790702170757714
variable x_5 ub 8.672524288611543
variable x_5 lb -0.790702170757714
Initial constraint:	 input LE: x_6 <= 4.24127975754059
variable x_6 ub 5.107769273886023
variable x_6 lb -2.472252836582964
Initial constraint:	 input GE: x_6 >= 4.24127975754059
variable x_6 ub 5.107769273886023
variable x_6 lb -2.472252836582

In [290]:
def find_ranges(minimal_explanation: list, epsilon: float):
    for constraint in minimal_explanation:
        print('Initial constraint:' + '\t', constraint)

        variable = constraint.lhs
        constraint_val = constraint.rhs
        if constraint.sense == docplex.mp.constants.ComparisonType.LE: #expand upperbound
            constraint.rhs = variable.ub
            print(f"New LE constraint: {constraint.rhs}")
            testing_model.minimize(variable)
            sol = testing_model.solve()
            if sol:
                print(f"Variable {variable} changes the class with value = {testing_model.objective_value}\n")
            else:
                print(f"Variable {variable} reached upper limit without changing class\n")
            #constraint.rhs = testing_model.objective_value - epsilon
            testing_model.remove_constraint(constraint)
            testing_model.add_constraint(constraint.lhs <= testing_model.objective_value - epsilon,'input LE')
        elif constraint.sense == docplex.mp.constants.ComparisonType.GE: #expand lowerbound
            constraint.rhs = variable.lb
            print(f"New GE constraint: {constraint.rhs}")
            testing_model.maximize(variable)
            sol = testing_model.solve()
            if sol:
                print(f"Variable {variable} changes the class with value = {testing_model.objective_value}\n")
            else:
                print(f"Variable {variable} reached lower limit without changing class\n")
            testing_model.remove_constraint(constraint)
            testing_model.add_constraint(constraint.lhs >= testing_model.objective_value + epsilon,'input GE')
        else:
            raise Exception('Constraint sense was neither LE nor GE')
            

In [291]:
find_ranges(linear_constraints, epsilon = 0.00001)
#linear_constraints = testing_model.find_matching_linear_constraints('input')
#linear_constraints
find_bounds(linear_constraints)

Initial constraint:	 input LE: x_0 <= 2.967691214515491
New LE constraint: 5.1279016612406805
Variable x_0 reached upper limit without changing class



DOcplexException: Model<Copy of Copy of docplex_model2> did not solve successfully

In [267]:
log_and_improve_explanation(linear_constraints, epsilon=0.01)

Initial constraint:	 input LE: x_0 <= 5.1279016612406805
Final constraint:	 input LE: x_0 <= 5.1279016612406805

Initial constraint:	 input GE: x_0 >= 5.1279016612406805
Final constraint:	 input GE: x_0 >= -2.3420983387592487

Initial constraint:	 input LE: x_0 <= -2.343651902203461
Final constraint:	 input LE: x_0 <= 5.126348097796468

Initial constraint:	 input GE: x_0 >= -2.343651902203461
Final constraint:	 input GE: x_0 >= -2.343651902203461

Initial constraint:	 input LE: x_3 <= 4.136918742833424
Final constraint:	 input LE: x_3 <= 4.136918742833424

Initial constraint:	 input GE: x_3 >= 4.136918742833424
Final constraint:	 input GE: x_3 >= -2.3330812571665267

Initial constraint:	 input LE: x_3 <= -2.335654020954433
Final constraint:	 input LE: x_3 <= 4.134345979045517

Initial constraint:	 input GE: x_3 >= -2.335654020954433
Final constraint:	 input GE: x_3 >= -2.335654020954433

Initial constraint:	 input LE: x_5 <= 8.672524288611543
Final constraint:	 input LE: x_5 <= 8.67252

In [268]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.1279016612406805),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-2.3420983387592487),
 docplex.mp.LinearConstraint[input LE](x_0,LE,5.126348097796468),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-2.343651902203461),
 docplex.mp.LinearConstraint[input LE](x_3,LE,4.136918742833424),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-2.3330812571665267),
 docplex.mp.LinearConstraint[input LE](x_3,LE,4.134345979045517),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-2.335654020954433),
 docplex.mp.LinearConstraint[input LE](x_5,LE,8.672524288611543),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.7874757113883172),
 docplex.mp.LinearConstraint[input LE](x_5,LE,8.669297829242147),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,5.107769273886023),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-2.4722307261139034),
 docplex.mp.LinearConstraint[input LE](x_6,LE,5.107747163416963)

In [269]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [270]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.1279016612406805),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-2.3420983387592487),
 docplex.mp.LinearConstraint[input LE](x_0,LE,5.126348097796468),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-2.343651902203461),
 docplex.mp.LinearConstraint[input LE](x_3,LE,4.136918742833424),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-2.3330812571665267),
 docplex.mp.LinearConstraint[input LE](x_3,LE,4.134345979045517),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-2.335654020954433),
 docplex.mp.LinearConstraint[input LE](x_5,LE,8.672524288611543),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.7874757113883172),
 docplex.mp.LinearConstraint[input LE](x_5,LE,8.669297829242147),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,5.107769273886023),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-2.4722307261139034),
 docplex.mp.LinearConstraint[input LE](x_6,LE,5.107747163416963)